## Project 4:

### Purpose

### Usage

Upload this notebook or DBC archive on databricks platform

---------------

##  Task1.  Extract 
#### a. Fetching The 7z archive

**Skip this Section if you already have performed the extraction process and jump to checkpoint for pulling data from split json files.**

In [0]:
# Checking if archive is downloaded in memory.
try:
    dbutils.fs.ls("file:/databricks/driver/dblp.v13.7z")
    print("Archive in filesystem (file:/databricks/driver/dblp.v13.7z)")
except:
    # If archive is not in memory, Checking databricks store for cached version and pulling into memory.
    try:
        dbutils.fs.ls("dbfs:/FileStore/data/dblp.v13.7z")
        print("Archive located in FileStore. Copying into local store..")
        dbutils.fs.cp("dbfs:/FileStore/data/dblp.v13.7z", "file:/databricks/driver/dblp.v13.7z")
        print("Completed")
    except:
        # If archive is not cached, downloading and storing in databricks store.
        print("7z archive not found. Fetching from URL...")
        !wget https://originalstatic.aminer.cn/misc/dblp.v13.7z
        print("7z archive Downloaded. Moving archive to FileStore..")
        dbutils.fs.mkdirs("dbfs:/FileStore/data")
        dbutils.fs.cp("file:/databricks/driver/dblp.v13.7z", "dbfs:/FileStore/data/dblp.v13.7z")
        print("Completed.")

In [0]:
# The returned array should have one object of FileInfo with size =2568255035

dbutils.fs.ls("file:/databricks/driver/dblp.v13.7z")

### b. Extracting Archive into json chunks

#### b1. Extracting 7zip file into json.

In [0]:
!pip install py7zr -q

In [0]:
import py7zr

archive = py7zr.SevenZipFile('dblp.v13.7z', mode='r')
archive.extractall()
archive.close()

In [0]:
dbutils.fs.ls("file:/databricks/driver/dblpv13.json")

#### b2. Cleaning NumberInt(#) tags

The json data contains non-confirming tags, and so cannot be parsed as it is. We will read each line and substitute the tag. (This should take about 25 minutes)

In [0]:
import re

# Cleaning the `NumberInt` tag
fin = open(f"dblpv13.json")
fout = open(f"dblpv13_clean.json", "wt")
for line in fin:
    fout.write(re.sub(r"NumberInt\([\d]*\)", lambda x: "".join(re.findall(r"\d", x.group(0))), line))
fin.close()
fout.close()

#### b3. Partitioning Dataset into JSON files
Since the whopping 16 GB of json data cannot be loaded into memory directly, we need to partition the data into smaller chunks (300k objects per chunk) for processing.  
We also parse data encoded as Decimal data with DecimalEncoder.

In [0]:
%mkdir data

In [0]:
import ijson
import json
import decimal

class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return str(o)
        return super(DecimalEncoder, self).default(o)

data_dir = 'data/'
with open('dblpv13_clean.json', 'r') as f:
    counter, file_id = 0, 0
    file_buffer = []
    for obj_data in ijson.items(f, 'item'):
        file_buffer.append(obj_data)
        counter += 1
        if counter % 300000 == 0:
            print(f" Saving, data_PART_{file_id}.json in {data_dir}")
            f = open(f'{data_dir}data_PART_{file_id}.json', 'w')
            dump = json.dumps(file_buffer, cls=DecimalEncoder)
            f.write(dump)
            f.close()
            file_id += 1
            file_buffer = []
f = open(f'{data_dir}data_PART_{file_id}.json', 'w')
dump = json.dumps(file_buffer, cls=DecimalEncoder)
print(f" Saving, data_PART_{file_id}.json in {data_dir}")
f.write(dump)
f.close()
file_id += 1
file_buffer = []

#### b4. Moving files to dbfs FileStore from instance storage. Building Checkpoint.

In [0]:
# removing old json stored in filestore.
dbutils.fs.rm("dbfs:/FileStore/data/split_data/", recurse = True)
# Creating dir to store json in filestore..
dbutils.fs.mkdirs("dbfs:/FileStore/data/split_data")
# confirming dir is empty
dbutils.fs.ls("dbfs:/FileStore/data/split_data")

In [0]:
# Copying all json parts into filestore.
dbutils.fs.cp("file:/databricks/driver/data/", "dbfs:/FileStore/data/split_data", recurse = True)

## Task2. NLP Unsupervised Learning

#### Goal: Read data from databricks Filestore into dataframes (Checkpoint after data load)

In [0]:
dbutils.fs.ls("dbfs:/delta/tables/partial/")

Out[1]: [FileInfo(path='dbfs:/delta/tables/partial/Author/', name='Author/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/delta/tables/partial/FactTable/', name='FactTable/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/delta/tables/partial/FieldOfStudy/', name='FieldOfStudy/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/delta/tables/partial/Language/', name='Language/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/delta/tables/partial/Organization/', name='Organization/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/delta/tables/partial/Publication/', name='Publication/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/delta/tables/partial/Venue/', name='Venue/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/delta/tables/partial/graphCheckPoint/', name='graphCheckPoint/', size=0, modificationTime=0)]

In [0]:
import uuid
from functools import reduce
from pyspark.sql import Row
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType, IntegerType
from typing import List
from pyspark.sql.functions import udf

# Here Path indicates input file path, and delta_dir points to file
path = "dbfs:/FileStore/data/split_data/"
delta_dir = "dbfs:/delta/tables/"

# There should be 18 files each with 300 k records. This would change if you change split value.
file_count = len(dbutils.fs.ls(path))
assert file_count == 18, "Data not found. You may want to check the path or run the notebook from start again. If you updated the split value, ignore this assertion error"

In [0]:
# Build map of spark dataframes by reading json partition chunk files
# dataframes_map = map(lambda r: spark.read.option("inferSchema", True).json(r), [f"{path}data_PART_{num}.json" for num in range(file_count)])
# reduce the dataframes into single dataframe by performing union over the mapped frames.
# union = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), dataframes_map)

# Reading first chunk for Testing
union = spark.read.option("inferSchema", True).json(f"{path}data_PART_0.json")

# jsonSchema = StructType([
# 		StructField("_id", StringType(), True),
# 		StructField("abstract", StringType(), True),
# 		StructField("authors", ArrayType(StructType([
# 			StructField("_id", StringType(), True),
# 			StructField("bio", StringType(), True),
# 			StructField("email", StringType(), True),
# 			StructField("gid", StringType(), True),
# 			StructField("name", StringType(), True),
# 			StructField("name_zh", StringType(), True),
# 			StructField("oid", StringType(), True),
# 			StructField("oid_zh", StringType(), True),
# 			StructField("orcid", StringType(), True),
# 			StructField("org", StringType(), True),
# 			StructField("org_zh", StringType(), True),
# 			StructField("orgid", StringType(), True),
# 			StructField("orgs", ArrayType(StringType(), True), True),
# 			StructField("orgs_zh", ArrayType(StringType(), True), True),
# 			StructField("sid", StringType(), True)
# 		]), True), True),
# 		StructField("doi", StringType(), True),
# 		StructField("fos", ArrayType(StringType(), True), True),
# 		StructField("isbn", StringType(), True),
# 		StructField("issn", StringType(), True),
# 		StructField("issue", StringType(), True),
# 		StructField("keywords", ArrayType(StringType(), True), True),
# 		StructField("lang", StringType(), True),
# 		StructField("n_citation", LongType(), True),
# 		StructField("page_end", StringType(), True),
# 		StructField("page_start", StringType(), True),
# 		StructField("pdf", StringType(), True),
# 		StructField("references", ArrayType(StringType(), True), True),
# 		StructField("title", StringType(), True),
# 		StructField("url", ArrayType(StringType(), True), True),
# 		StructField("venue", StructType([
# 			StructField("_id", StringType(), True),
# 			StructField("issn", StringType(), True),
# 			StructField("name", StringType(), True),
# 			StructField("name_d", StringType(), True),
# 			StructField("name_s", StringType(), True),
# 			StructField("online_issn", StringType(), True),
# 			StructField("publisher", StringType(), True),
# 			StructField("raw", StringType(), True),
# 			StructField("raw_zh", StringType(), True),
# 			StructField("sid", StringType(), True),
# 			StructField("src", StringType(), True),
# 			StructField("t", StringType(), True),
# 			StructField("type", LongType(), True)
# 		]), True),
# 		StructField("volume", StringType(), True),
# 		StructField("year", LongType(), True)
# 	])


# union = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(path)

union = union.na.drop(subset=["authors"])
union = union.dropDuplicates(["_id"])
union = union.filter(union.lang == 'en')
union = union.na.drop(subset=["title", "abstract"])
union.printSchema()

root
 |-- _id: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- name_zh: string (nullable = true)
 |    |    |-- oid: string (nullable = true)
 |    |    |-- oid_zh: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- org_zh: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- orgs_zh: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- sid: string (nullable = true)
 |-- doi: string (nulla

In [0]:
# union = spark.read.format('delta').load('dbfs:/delta/tables/partial/Publication/')
# union = union.dropDuplicates(["_id"])
union.show()

+--------------------+--------------------+--------------------+--------------------+----+-------------+--------------------+-----+--------------------+----+----------+------------+------------+--------------------+----------+-------------+--------------------+--------------------+-------------+----+
|                 _id|            abstract|             authors|                 doi| fos|         isbn|                issn|issue|            keywords|lang|n_citation|    page_end|  page_start|                 pdf|references|        title|                 url|               venue|       volume|year|
+--------------------+--------------------+--------------------+--------------------+----+-------------+--------------------+-----+--------------------+----+----------+------------+------------+--------------------+----------+-------------+--------------------+--------------------+-------------+----+
|53e99784b7602d970...|360° represents t...|[{53f46946dabfaec...|10.1145/1665137.1...|null|    

#### Cleaning bad records (empty author lists, small titles)

In [0]:
# Deleting entries with small Titles (less than 3 words) and empty author list
text_size_ = udf(lambda s: len(s.split()), IntegerType())
arr_len_ = udf(lambda s: len(s), IntegerType())
union = union.filter(text_size_(F.col("Title")) > 3)

In [0]:
# TODO; exploratory data analysis

### Data preprocessing

##### https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79

In [0]:
!pip install spacy unidecode contractions nltk tqdm -q
!spacy download en_core_web_md

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
     |███▍                            | 3.6 MB 7.0 

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 33.5 MB 5.3 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [0]:
custom_stopwords = ['doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure','rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 'al.', 'Elsevier', 'PMC', 'CZI', 'www', 'i.e.', 'copyrights', 'peers', 'authors', 's', 'index', 'table', 'present', 'paper', 'papers']

In [0]:
import re
import spacy
import nltk
from tqdm import tqdm
import unidecode
import contractions
from nltk.stem import WordNetLemmatizer
from pyspark.ml.feature import Word2Vec
from pyspark.ml.clustering import KMeans

en = spacy.load('en_core_web_md')
sw_spacy = set(en.Defaults.stop_words)

nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

sw_spacy = list(sw_spacy) + custom_stopwords

numIterations = 200

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [0]:
def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text

def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text

def preprocess_udf():
    def preprocess(title, abstract):
        tokens = []
        # cleaning symbols
        if title:
            title = re.sub(r"[^A-Za-z]+", " ", title)
            title = remove_accented_chars(title)
            title = expand_contractions(title)
            tokens = [lemmatizer.lemmatize(word.lower()) for word in title.split() if word.lower() not in sw_spacy]
        if abstract:
            # cleaning symbols
            abstract = re.sub(r"[^A-Za-z ]+", "", abstract)
            # removing accented characters (required?)
            abstract = remove_accented_chars(abstract)
            abstract = expand_contractions(abstract)
            tokens += [lemmatizer.lemmatize(word.lower()) for word in abstract.split() if word.lower() not in sw_spacy]
        return tokens
    return udf(preprocess, ArrayType(StringType()))

In [0]:
union = union.withColumn("tokens", preprocess_udf()("abstract", "title"))

documents = union.select("_id", "tokens").filter(arr_len_(F.col("tokens")) > 0)

In [0]:
display(documents)

_id,tokens
53e99792b7602d9701f5b114,"List(positive, feedback, pilot, system, second, loop, control, feedforward, power, amplifier)"
53e99792b7602d9701f5b191,"List(human, sensibility, ergonomics, hse, apprehend, human, sensitivity, feature, measuring, human, sens, developing, table, related, psychology, physiology, main, purpose, hse, developing, human, centered, good, environment, relevant, technology, improved, life, quality, order, achieve, goal, test, bed, simulator, useful, tool, controlling, monitoring, physical, environment, deal, requirement, design, concept, specification, computing, environment, hse, hse, technology, development, program, sponsored, korean, ministry, science, technology, integrated, computing, system, composed, real, time, non, real, time, environment, non, real, time, development, environment, comprises, pc, window, nt, graphical, user, interface, coded, microsoft, visual, c, pc, independently, control, monitor, thermal, light, audio, video, environment, software, database, developed, non, real, time, environment, directly, ported, real, time, environment, local, area, network, real, time, computing, system, based, cpci, bus, control, integrated, hse, environment, collect, necessary, information, cpci, computing, system, composed, pentium, cpu, board, dedicated, o, board, quantity, determined, expandability, considered, integrated, computing, environment, hse, simulator, guarantee, real, time, capability, stability, expandability, hardware, maximize, portability, compatibility, maintainability, software, requirement, analysis, design, computational, environment, hse, humansensibility, ergonomics, simulator)"
53e99792b7602d9701f5b2d3,"List(present, novel, ia, instrumentation, amplifier, design, implantable, biomedical, device, system, db, cmrr, common, mode, rejection, ratio, proposed, ia, composed, stage, including, preamplifier, nd, order, bpf, band, pas, filer, dc, level, shifter, output, buffer, stage, low, noise, gm, c, amplifier, preamplifier, stage, reduce, coupled, thermal, noise, overwhelm, weak, neural, signal, bpf, designed, based, ota, operational, transconductance, amplifier, dual, current, switch, aiming, low, power, low, noise, demand, source, follower, employed, carry, dc, level, shifter, output, buffer, provides, output, signal, adequate, drive, following, stage, usually, adc, analog, digital, converter, detailed, analysis, proposed, circuitry, derived, solidify, proposed, architecture, proposed, design, implemented, tsmc, mum, p, m, cmos, process, result, post, layout, simulation, verify, performance, design, cmrr, better, db, important, input, noise, rms, merely, db, pvt, process, supply, voltage, temperature, corner, db, cmrr, lownoise, instrumentation, amplifier, neural, signal, sensing)"
53e99792b7602d9701f5b391,"List(prototyping, development, environment, graphical, interactive, telematic, application)"
53e99792b7602d9701f5b3e5,"List(augmented, reality, technology, overlay, virtual, image, information, generated, computer, real, scene, technology, combine, virtual, object, real, world, put, forward, registration, method, multi, marker, augmented, system, paleontology, magic, book, designed, realized, book, special, virtual, education, interactive, real, time, end, result, showed, study, prosperous, future, augmented, reality, applying, education, seen, method, multiplemarker, register, application, virtual, education)"
53e99792b7602d9701f5b3f3,"List(abstract, industrial, research, project, demonstrated, feasibility, applying, category, theoretic, method, specification, synthesis, maintenance, industrial, strength, software, system, demonstration, kind, tool, purpose, kestrel, specware, software, development, system, describe, experience, discus, broadening, application, category, theoretic, method, industry, technology, promising, need, additional, development, generally, usable, surprising, given, mathematical, foundation, hand, believe, demonstration, turning, point, use, math

In [0]:
# word2vec = Word2Vec()
# model = word2vec.fit(documents.rdd.map(lambda x: x.tokens).collect())

word2Vec = Word2Vec(inputCol="tokens", outputCol="features").fit(documents)
documents = word2Vec.transform(documents)

In [0]:
display(documents)

_id,tokens,features
53e99792b7602d9701f5b114,"List(positive, feedback, pilot, system, second, loop, control, feedforward, power, amplifier)","Map(vectorType -> dense, length -> 100, values -> List(0.0010927247814834118, -0.048946963064372545, 0.25271623916924, -0.10146973873488606, 0.0929327979683876, 0.0620644030161202, 0.18247236199676992, -0.008089794171974063, 0.011958000995218755, -3.759678453207016E-4, -0.07504244782030584, -0.1550638496875763, -0.05285673774778843, 0.07350046783685685, 0.15567383375018837, 0.22132079489529133, -0.19899859167635442, -0.12370968582108617, 0.032582087069749834, 0.05979675310663879, 0.14004724696278573, -0.17189322207123042, -0.009475089702755214, -0.005496566742658616, -0.09328386827837676, -0.00999482460319996, -0.039211823139339685, 0.026010511815547945, 0.049516450613737106, 0.0878322945907712, -0.11589562557637692, -0.16261781491339208, -0.07318642595782876, 0.14487319104373456, -0.07714137155562639, 0.05137619879096747, -0.051670830836519605, -0.09527750127017498, -0.017030235566198825, -0.23367526158690455, 0.002070311829447746, -0.0012378905434161426, -0.14453946468420328, -0.0705340939341113, -0.12340338854119182, -0.0851744070649147, 0.011616210453212261, 0.08518979027867318, -0.15146196018904448, 0.0819782167673111, 0.043000865587964655, -0.022688549198210242, -0.02991621606051922, 0.016066246293485166, 0.08565911119803787, 0.13398654172196986, 0.10416028469335288, 0.13412011228501797, 0.11769673302769662, -0.12895780727267267, 0.05083277886733413, -0.08135408144444228, -0.017674032598733902, -0.14100532587617637, 0.1559406191110611, 0.0865834909491241, -0.08153893876587973, -0.018087121285498144, 0.05385218560695648, -0.19007823567371818, -0.0645210187882185, -0.027516840398311617, -0.19985381439328195, -0.0546268779784441, -0.07770143551751972, 0.023293661139905456, 0.04728543199598789, 0.07786396192386746, 0.07273276895284653, -0.062126229703426364, 0.11661941409111024, 0.07477662162855268, 0.04439299148507417, 0.07757203783839942, -0.01541152037680149, -0.010049501061439516, 0.016276248078793288, -0.3513743249815889, 0.20911250058561565, 0.09275583177804947, -0.25134466248564424, -0.01449076607823372, -0.09569526314735413, -0.18853178117424252, 0.040441786218434576, 0.18679111078381538, 0.07140682929893956, -0.16679169202689081, 0.06341372523456812, -0.11312212012708188))"
53e99792b7602d9701f5b191,"List(human, sensibility, ergonomics, hse, apprehend, human, sensitivity, feature, measuring, human, sens, developing, table, related, psychology, physiology, main, purpose, hse, developing, human, centered, good, environment, relevant, technology, improved, life, quality, order, achieve, goal, test, bed, simulator, useful, tool, controlling, monitoring, physical, environment, deal, requirement, design, concept, specification, computing, environment, hse, hse, technology, development, program, sponsored, korean, ministry, science, technology, integrated, computing, system, composed, real, time, non, real, time, environment, non, real, time, development, environment, comprises, pc, window, nt, graphical, user, interface, coded, microsoft, visual, c, pc, independently, control, monitor, thermal, light, audio, video, environment, software, database, developed, non, real, time, environment, directly, ported, real, time, environment, local, area, network, real, time, computing, system, based, cpci, bus, control, integrated, hse, environment, collect, necessary, information, cpci, computing, system, composed, pentium, cpu, board, dedicated, o, board, quantity, determined, expandability, considered, integrated, computing, environment, hse, simulator, guarantee, real, time, capability, stability, expandability, hardware, maximize, portability, compatibility, maintainability, software, requirement, analysis, design, computational, environment, hse, humansensibility, ergonomics, simulator)","Map(vectorType -> dense, length -> 100, values -> List(0.060469724318403514, 0.012265218924845993, 0.0

In [0]:
costs = {}

for k in tqdm(range(30, 65, 10)):
    kmeans_model = KMeans(maxIter=numIterations, k=k, seed=100).fit(documents)
    costs[k] = kmeans_model.summary.trainingCost
    print(k, costs[k])

 50%|█████     | 2/4 [54:12<54:27, 1633.76s/it]  40 83780.20496213346


In [0]:
costs

In [0]:
import matplotlib.pyplot as plt

plt.plot(list(range(15, 45, 3)), costs)
plt.xlabel("Clusters")
plt.ylabel(TrainingCost)
plt.title("Elbow plot")
plt.show()

In [0]:
# saving best model
import numpy as np
from pyspark.ml.clustering import KMeansModel

best_k = np.argmin(costs)
kmeans_models[best_k].save("dbfs:/FileStore/data/clustering_model")

In [0]:

best_model = KMeansModel.load("dbfs:/FileStore/data/clustering_model")
predictions = best_model.transform(documents)
predictions.show()